In [1]:
import pandas as pd
import geopandas as gpd
import sqlalchemy as sa
import pyodbc
import tkinter as tk
from sqlalchemy import create_engine
from api_config import geocode_postcode, fetch_isochrone, PostcodeNotFound
from geo_config import isochrone_to_gdf, teams_to_gdf, filter_teams_by_minutes,list_business_units, apply_team_filters

In [ ]:
# SQL server details 
server = 'dblistener1'
database = 'information_centre'
driver = 'ODBC Driver 17 for SQL Server' # Create SQLAlchemy engine and install necessary drivers
connection_string = f'mssql+pyodbc://{server}/{database}?driver={driver}'
engine = create_engine(connection_string)

# SQL query to select dataset from SSMS tables and save into allvisits dataframe
query = """
SELECT
    a.intContractorID,
    a.strName AS Contractor,
    a.strMobileTel AS MobileTel,
    a.strWebsite AS Email,
    e.BusinessUnitID,
    e.Name AS BusinessUnit,
    UPPER(a.strPostcode) AS Postcode,
    g.Latitude,
    g.Longitude,
    a.InternalContractor
FROM
    tblContractor a
    LEFT JOIN Contractor_Business_Unit d
        ON a.intContractorID = d.ContractorID
    LEFT JOIN Business_Unit e
        ON d.BusinessUnitID = e.BusinessUnitID
    LEFT JOIN Business_Unit_Master_Status f
        ON d.StatusID = f.StatusID
    LEFT JOIN dbs_PostCode.dbo.tblPostcodes_New g
        ON REPLACE(a.strPostcode, ' ', '') = g.PostcodeNoSpaces COLLATE Latin1_General_CI_AS
WHERE
    ISNULL(a.bDisabled, 0) = 0
    AND f.StatusID IN (60, 70, 80)
    AND ISNULL(a.IsTest, 0) = 0
    AND e.BusinessUnitID != 37;
"""
with engine.connect() as connection:
    fieldteams = pd.read_sql(query, connection)
    connection.close()

In [ ]:
try:
    site_lon, site_lat = geocode_postcode("CM12 0EQ")
except PostcodeNotFound as e:
    print(f"Postcode error: {e}")
else:
    # One API call to Mapbox Isochrone API
    iso_geojson = fetch_isochrone(site_lon, site_lat)

    # Filter locally using GeoPandas rather than multiple API calls
    iso_gdf   = isochrone_to_gdf(iso_geojson)
    teams_gdf = teams_to_gdf(fieldteams)

    # Filter teams within the 60-minute isochrone to reduce multiple API calls later
    filtered_teams = filter_teams_by_minutes(teams_gdf, iso_gdf, minutes=60)

# Business Unit options for dropdown
bu_options = list_business_units(filtered_teams)

# Example selections
chosen_bu = "Grounds Maintenance"
chosen_internal_flag = None # 1 = Directly Employed, 0 = Contractor, None = both

# Apply filters without making any OSRM calls
teams_ready_for_routing = apply_team_filters(
    filtered_teams,
    business_unit=chosen_bu,
    internal_flag=chosen_internal_flag
)

In [ ]:
launch_gui(fieldteams)